In [3]:
import numpy as np
import pandas as pd
from matplotlib import rc
rc("animation", html="jshtml")

import importlib
import models
importlib.reload(models)
import tools
importlib.reload(tools)

from tools import (
    animate_week_play,
    get_din_dout,
    test_frame_alignment,
    visualize_predictions,
)
from models import (
    KinematicModel,
    train_eval_model,
    KinematicBoundaryModel,
    BayesianKinematicModel,
    HierarchicalBayesianKinematicModel,
    BayesianKinematicHMM,
    fit_model_up_to_week,
    train_eval_until_week,
)


In [4]:
def get_game_play_index(input_path):
    df = pd.read_csv(input_path, usecols=["game_id", "play_id"])
    df = df.drop_duplicates().sort_values(["game_id", "play_id"])
    return df


In [5]:
week = 6  # 18 weeks
index_df = get_game_play_index(f"train/input_2023_w{week:02d}.csv")
print(f"{index_df.shape[0]} plays for week {week}")
idx = 7
gi = index_df.iloc[idx].game_id
pi = index_df.iloc[idx].play_id
print(f"Selected: game_id={gi}, play_id={pi}")


793 plays for week 6
Selected: game_id=2023101200, play_id=589


# Hierarchical Bayesian Kinematic Model

This model extends the basic Bayesian kinematic model by adding hierarchical structure:
- Position-specific noise scales (WRs, RBs, CBs may have different movement patterns)
- Player-specific parameters (nested in position)
- Global priors that share information across all players/positions


In [ ]:
# Fit Hierarchical Bayesian Model
# Speedup options (uncomment to use):
#   - max_samples=100000: Subsample to 100k rows (8x faster, slight accuracy loss)
#   - use_vi=True: Use Variational Inference (10-50x faster, less accurate than MCMC)
#   - draws=200, tune=200: Reduce iterations (2-3x faster, less robust)
#   - chains=1: Single chain (2x faster, less diagnostics)
#   - use_position_hierarchy=False: Disable hierarchy (faster, less flexible)

hierarchical_model = HierarchicalBayesianKinematicModel(fps=10.0, use_accel=True)
hierarchical_model = fit_model_up_to_week(
    hierarchical_model,
    max_train_week=3,
    source="input",   # IMPORTANT: needs s,a,dir
    use_position_hierarchy=True,
    use_player_hierarchy=False,  # Set to True for player-specific (more expensive)
    # Speedup options (uncomment to use):
    # max_samples=100000,  # Subsample to 100k rows (much faster)
    # use_vi=True,  # Use Variational Inference (10-50x faster)
    # vi_n=5000,  # VI iterations if use_vi=True
    draws=500,  # Reduce to 200 for faster training
    tune=500,   # Reduce to 200 for faster training
    target_accept=0.9,
    chains=2,   # Reduce to 1 for faster training
)


[fit_model_up_to_week] Training on weeks: [1, 2, 3]


Building step df (input):   0%|          | 0/3 [00:00<?, ?week/s]

[fit_model_up_to_week] Fitting model 'hierarchical_bayes_kinematic' on 840,437 rows...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_x_global, sigma_y_global, sigma_x_pos, sigma_y_pos, bias_x_pos, bias_y_pos]


Output()

In [ ]:
# Visualize with heatmaps showing probability density
ani_hierarchical = visualize_predictions(
    hierarchical_model,
    week=week,
    game_id=gi,
    play_id=pi,
    horizon=3,
    interval=120,
    pause_time=None,
    bayes_samples=300,  # More samples for better heatmap
    show_paths=True,
    show_cones=True,
    show_heatmaps=True,  # Enable heatmaps!
    heatmap_alpha=0.5,
    heatmap_grid_resolution=60,
    heatmap_bandwidth=0.8,
    show_legend=True,
)
ani_hierarchical


# Hidden Markov Model (HMM) for Movement Regimes

This model assumes players switch between discrete movement regimes:
- Different states have different kinematic parameters
- State probabilities depend on player features (speed, acceleration)
- Captures behavioral switches (e.g., route running → ball tracking)


In [ ]:
# Fit HMM Model
hmm_model = BayesianKinematicHMM(
    fps=10.0,
    use_accel=True,
    n_states=3,  # Number of movement regimes
    state_names=["route_running", "ball_tracking", "evasive"],
)
hmm_model = fit_model_up_to_week(
    hmm_model,
    max_train_week=3,
    source="input",
    group_cols=["game_id", "play_id", "nfl_id"],  # Group by player sequences
    draws=500,
    tune=500,
    target_accept=0.9,
    chains=2,
)


In [ ]:
# Visualize HMM with heatmaps
ani_hmm = visualize_predictions(
    hmm_model,
    week=week,
    game_id=gi,
    play_id=pi,
    horizon=3,
    interval=120,
    pause_time=None,
    bayes_samples=300,
    show_paths=True,
    show_cones=True,
    show_heatmaps=True,  # Enable heatmaps!
    heatmap_alpha=0.5,
    heatmap_grid_resolution=60,
    heatmap_bandwidth=0.8,
    show_legend=True,
)
ani_hmm


## Compare Models Side-by-Side

You can tab through the animations above to see:
1. **Hierarchical Model**: Position-specific uncertainty, shared information across similar players
2. **HMM Model**: Regime-switching behavior, captures different movement modes

The heatmaps show the probability density of where players could end up - warmer colors indicate higher probability regions.


In [ ]:
# Compare with basic Bayesian model (no heatmaps for comparison)
basic_bayes = BayesianKinematicModel(fps=10.0, use_accel=True)
basic_bayes = fit_model_up_to_week(
    basic_bayes,
    max_train_week=3,
    source="input",
    draws=500,
    tune=500,
    target_accept=0.9,
    chains=2,
)

ani_basic = visualize_predictions(
    basic_bayes,
    week=week,
    game_id=gi,
    play_id=pi,
    horizon=3,
    interval=120,
    pause_time=None,
    bayes_samples=300,
    show_paths=True,
    show_cones=True,
    show_heatmaps=True,  # Also show heatmaps for basic model
    heatmap_alpha=0.5,
    heatmap_grid_resolution=60,
    heatmap_bandwidth=0.8,
    show_legend=True,
)
ani_basic
